In [ ]:
import sys
if '../' not in sys.path:
    sys.path.append('../')

In [ ]:
import random
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from simulab.simulation.core.runner import Runner
from simulab.simulation.core.lattice import Lattice
from simulab.simulation.core.neighborhood import ExpandedMoore
from simulab.simulation.core.experiment import ExperimentParametersSet
from simulab.simulation.core.equilibrium_criterion import WithoutCriterion

from src.market import Market, Producer, Consumer

from simulab.simulation.plotters.final_grid import FinalGridSeries
from simulab.simulation.plotters.numerical_series import NumericalSeries
from simulab.simulation.plotters.categorical_animated_lattice import (
    CategoricalAnimatedLatticeSeries,
)

random.seed(1000)
np.random.seed(1000)

In [ ]:
# Parámetros

length = 20
capital = 1_000
stock = 5_000_000_000
producer_probability = 0.25
profit_period = 7
price_ratio = (1.2, 1.5)
fixed_cost = (10, 1)
marginal_cost = (10, 1)
quantity_to_buy = (1, 0)
criterion = WithoutCriterion()

In [ ]:
# Funciones auxiliares

def create_configuration():
    configuration = Lattice.with_probability(producer_probability, length)

    for i in range(length):
        for j in range(length):
            agent_type = configuration.at(i, j)
            if agent_type == Consumer.TYPE:
                agent = Consumer()
            else:
                _marginal_cost = abs(np.random.normal(*marginal_cost))
                _price_ratio = np.random.uniform(*price_ratio)
                agent = Producer(
                    capital=capital,
                    stock=stock,
                    price=_marginal_cost * _price_ratio,
                    fixed_cost=abs(np.random.normal(*fixed_cost)),
                    marginal_cost=_marginal_cost,
                    profit_period=profit_period,
                )
            configuration.set(i, j, _with=agent)
    return configuration

def add_whale_to(configuration, position):
    configuration.set(position[0], position[1], _with=Producer(
        capital=capital*5,
        stock=stock,
        price=3,
        fixed_cost=2,
        marginal_cost=2,
        profit_period=profit_period))

def parameters_with(configuration, bankrupt_enabled=False):
    return ExperimentParametersSet(
        length=[length],
        neighborhood=[ExpandedMoore(3)],
        agent_types=[2],
        capital=[capital],
        producer_probability=[producer_probability],
        profit_period=[profit_period],
        price_ratio=[price_ratio],
        fixed_cost=[fixed_cost],
        marginal_cost=[marginal_cost],
        quantity_to_buy=[quantity_to_buy],
        bankrupt_enabled=[bankrupt_enabled],
        configuration=[configuration],
    )

def bankrupted_on(runner):
    producer_positions = set(runner.experiments[0]._by_type[1])
    return [runner.experiments[0].configuration.at(*position).capital <= 0
            for position in producer_positions]

def execute_with(configuration, data, strategy_name, max_steps=1000, bankrupt_enabled=False):
    params = parameters_with(configuration, bankrupt_enabled=bankrupt_enabled)
    runner = Runner(Market, params, criterion, max_steps=max_steps)
    runner.start()
    producer_positions = set(runner.experiments[0]._by_type[1])
    producers_amount = len(producer_positions)
    current_bankrupted = bankrupted_on(runner).count(True)
    data["Capital"] = data["Capital"] + ["sin capital", "con capital"]
    data["Cantidad"] = data["Cantidad"] + [current_bankrupted, producers_amount-current_bankrupted]
    data["Estrategia"] = data["Estrategia"] + ([strategy_name] * 2)
    return producer_positions, runner

In [ ]:
configuration = create_configuration()
experiment_parameters_set = ExperimentParametersSet(
    length=[length],
    neighborhood=[ExpandedMoore(3)],
    agent_types=[2],
    capital=[capital],
    producer_probability=[producer_probability],
    profit_period=[profit_period],
    price_ratio=[price_ratio],
    fixed_cost=[fixed_cost],
    marginal_cost=[marginal_cost],
    quantity_to_buy=[quantity_to_buy],
    bankrupt_enabled=[False, True],
    configuration=[configuration],
)
runner0 = Runner(Market, experiment_parameters_set, criterion, max_steps=1500)
runner0.start()

In [ ]:
FinalGridSeries.show_up(
    "price_lattice", 
    runner=runner0,
    plot_title="Evolución de los precios de Mercado",
    leyend="Precio",
    colorscale="Viridis",
)

In [ ]:
for experiment_id in [0, 1]:
    CategoricalAnimatedLatticeSeries.show_up(
        "profit_categorized_lattice",
        runner=runner0,
        experiment_id=experiment_id,
        plot_title=f"Evolución de la ganancia de los productores (Experimento nº{experiment_id})",
        height=500,
    )

In [ ]:
for experiment_id in [0, 1]:
    CategoricalAnimatedLatticeSeries.show_up(
        "capital_categorized_lattice",
        runner=runner0,
        experiment_id=experiment_id,
        plot_title=f"Evolución del capital de los productores (Experimento nº{experiment_id})",
        height=500,
    )

In [ ]:
# config = runner0.experiments[0].configuration
# add_whale_to(config,)

In [ ]:
repetitions = 10
max_steps = 1500

data = {}
data["Capital"] = [] 
data["Cantidad"] = []
data["Estrategia"] = []

for _ in range(repetitions):
    configuration = create_configuration()

    positions, runner1 = execute_with(configuration, data, "sin bancarrota", max_steps=max_steps, bankrupt_enabled=False)
    positions, runner2 = execute_with(configuration, data, "con bancarrota", max_steps=max_steps, bankrupt_enabled=True)
    positions = iter(positions)
    add_whale_to(configuration, next(positions))
    add_whale_to(configuration, next(positions))
    positions, runner3 = execute_with(configuration, data, "bancarrota + ballena", max_steps=max_steps, bankrupt_enabled=True)

In [ ]:
data = pd.DataFrame(pd.DataFrame(data), columns=["Capital", "Cantidad", "Estrategia"])
fig = px.bar(data, x="Estrategia", y="Cantidad", color="Capital", title="Cantidad de productores según su capital y estrategia de simulación")
fig.show()